Tested on an Ubuntu 16.04 machine with two Intel Xeon E5-2630 processors (12 true cores) and 192 GB RAM (PyGWF uses ~60 GB to decompress the raw frame with multiprocessing and the default number of processes). 

There are two steps to reading data from a .gwf file: (1) parse its header and structure information to determine where data is located, the compression types, etc. and (2) decompress the actual channel (or "FrVect") data. These are implemented in PyGWF as `parse_gwf_data` and `decompress_frvect`.

`parse_gwf_data` returns two dicts, the first describing every structure class defined in the file and the second containing the (possibly compressed) data for each instance of each class in the file. An instance of the FrVect class can then be passed to decompress_frvect to extract the channel data. 

`parse_gwf_data` must read the entire file sequentially and is not parallelized; it can take several minutes for a large frame file. Calls to `decompress_frvect` can be parallelized for multiple FrVect instances.

`get_frvects_from_gwf` takes a path to a frame file and (optionally) a list of channels. It first calls `parse_gwf_data`, then calls `decompress_frvect` in parallel with Python's multiprocessing module to decompress the FrVect instances corresponding to the specified channels. Results of the call to `parse_gwf_data` aren't retained, so every call to `get_frvects_from_gwf` includes the overhead of a call to `parse_gwf_data`. In practice it would be unnecessarily inefficient to call `get_frvects_from_gwf` repeatedly on the same frame file, but we do so below to illustrate the full amount of time required to decompress varying numbers of channels from a frame file from scratch.

Frame file specification: https://dcc.ligo.org/public/0000/T970130/002/T970130-v2.pdf

In [1]:
import subprocess
import random
import numpy as np
import pycbc.frame

import pygwf

print(pycbc.__version__)

1.15.4


In [2]:
!curl -O -J -L https://losc.ligo.org/s/events/LVT151012/H-H1_LOSC_4_V2-1128678884-32.gwf

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   266  100   266    0     0    548      0 --:--:-- --:--:-- --:--:--   548
100 1004k  100 1004k    0     0   713k      0  0:00:01  0:00:01 --:--:-- 4430k


In [3]:
hoft1_path = "H-H1_LOSC_4_V2-1128678884-32.gwf"
hoft2_path = "L-L1_HOFT_C00-1238900736-4096.gwf"
raw1_path = "L-L1_R-1238900736-64.gwf"

hoft1_start = 1128678884
hoft1_len = 32
hoft2_start = 1238900736
hoft2_len = 4096
raw1_start = 1238900736
raw1_len = 64

hoft1_strain_ch = "H1:LOSC-STRAIN"
hoft2_strain_ch = "L1:GDS-CALIB_STRAIN"

In [4]:
def get_channels_info(gwf_path):
    channel_names = []
    output = subprocess.check_output(args=["FrChannels", gwf_path]).decode().splitlines()
    for line in output:
        ch, _ = line.split(" ")
        channel_names.append(ch)
    return channel_names

In [5]:
def arrs_eq_nan(arr1, arr2):
    return np.all(np.logical_or(
        arr1 == arr2,
        np.logical_and(np.isnan(arr1), np.isnan(arr2))
    ))

In [6]:
%time hoft1_strain_pycbc = pycbc.frame.read_frame(hoft1_path, [hoft1_strain_ch], hoft1_start, hoft1_start + hoft1_len)

CPU times: user 100 ms, sys: 4 ms, total: 104 ms
Wall time: 105 ms


In [7]:
%time hoft1_strain_pygwf = pygwf.get_frvects_from_gwf(hoft1_path, [hoft1_strain_ch])

CPU times: user 72 ms, sys: 32 ms, total: 104 ms
Wall time: 102 ms


In [8]:
arrs_eq_nan(np.array(hoft1_strain_pycbc[0]._data), hoft1_strain_pygwf[hoft1_strain_ch])

True

In [9]:
%time hoft2_strain_pycbc = pycbc.frame.read_frame(hoft2_path, [hoft2_strain_ch], hoft2_start, hoft2_start + hoft2_len)
# Bug(?) in pycbc.frame.read_frame(): if you don't specify the start time and duration, you only get the first second of data. Probably related to this .gwf file containing multiple individual frames (each one second long).

CPU times: user 2.44 s, sys: 444 ms, total: 2.88 s
Wall time: 2.87 s


In [10]:
%time hoft2_strain_pygwf = pygwf.get_frvects_from_gwf(hoft2_path, [hoft2_strain_ch])

CPU times: user 1min 45s, sys: 11 s, total: 1min 56s
Wall time: 2min 42s


In [11]:
arrs_eq_nan(np.array(hoft2_strain_pycbc[0]._data), hoft2_strain_pygwf[hoft2_strain_ch])

True

In [12]:
%time hoft2_chs = get_channels_info(hoft2_path)

CPU times: user 0 ns, sys: 108 ms, total: 108 ms
Wall time: 293 ms


In [13]:
%time hoft2_all_pycbc = pycbc.frame.read_frame(hoft2_path, hoft2_chs, start_time=hoft2_start, duration=hoft2_len)

CPU times: user 9.02 s, sys: 1.71 s, total: 10.7 s
Wall time: 10.7 s


In [14]:
%time hoft2_all_pygwf = pygwf.get_frvects_from_gwf(hoft2_path)

CPU times: user 2min 30s, sys: 18.4 s, total: 2min 49s
Wall time: 4min 58s


In [15]:
all(arrs_eq_nan(np.array(hoft2_all_pycbc[i]._data), hoft2_all_pygwf[ch]) for i, ch in enumerate(hoft2_chs))

True

In [16]:
%time raw1_chs_all = get_channels_info(raw1_path)

raw1_chs_10000 = random.sample(raw1_chs_all, 10000)
raw1_chs_1000 = random.sample(raw1_chs_10000, 1000)
raw1_chs_100 = random.sample(raw1_chs_1000, 100)

CPU times: user 196 ms, sys: 176 ms, total: 372 ms
Wall time: 11 s


In [17]:
%time raw1_100_pycbc = pycbc.frame.read_frame(raw1_path, raw1_chs_100)

CPU times: user 36.6 s, sys: 1.26 s, total: 37.8 s
Wall time: 37.8 s


In [18]:
%time raw1_100_pygwf = pygwf.get_frvects_from_gwf(raw1_path, raw1_chs_100)

CPU times: user 2min 7s, sys: 11.2 s, total: 2min 18s
Wall time: 2min 21s


In [19]:
all(arrs_eq_nan(np.array(raw1_100_pycbc[i]._data), raw1_100_pygwf[ch]) for i, ch in enumerate(raw1_chs_100))

True

In [20]:
%time raw1_1000_pycbc = pycbc.frame.read_frame(raw1_path, raw1_chs_1000, raw1_start, raw1_start + raw1_len)

CPU times: user 6min 4s, sys: 12.5 s, total: 6min 16s
Wall time: 6min 16s


In [21]:
%time raw1_1000_pygwf = pygwf.get_frvects_from_gwf(raw1_path, raw1_chs_1000)

CPU times: user 2min 7s, sys: 10.6 s, total: 2min 18s
Wall time: 2min 30s


In [22]:
all(arrs_eq_nan(np.array(raw1_1000_pycbc[i]._data), raw1_1000_pygwf[ch]) for i, ch in enumerate(raw1_chs_1000))

True

In [23]:
%time raw1_10000_pycbc = pycbc.frame.read_frame(raw1_path, raw1_chs_10000, raw1_start, raw1_start + raw1_len)

CPU times: user 1h 4min 8s, sys: 1min 57s, total: 1h 6min 6s
Wall time: 1h 6min 4s


In [24]:
%time raw1_10000_pygwf = pygwf.get_frvects_from_gwf(raw1_path, raw1_chs_10000)

CPU times: user 2min 11s, sys: 9.7 s, total: 2min 20s
Wall time: 2min 51s


In [25]:
all(arrs_eq_nan(np.array(raw1_10000_pycbc[i]._data), raw1_10000_pygwf[ch]) for i, ch in enumerate(raw1_chs_10000))

True

In [26]:
%time raw1_all_pycbc = pycbc.frame.read_frame(raw1_path, raw1_chs_all, raw1_start, raw1_start + raw1_len)

CPU times: user 1d 19h 2min 32s, sys: 1h 29s, total: 1d 20h 3min 2s
Wall time: 1d 20h 2min 3s


In [27]:
%time raw1_all_pygwf = pygwf.get_frvects_from_gwf(raw1_path)

CPU times: user 3min 28s, sys: 45.1 s, total: 4min 13s
Wall time: 12min 21s


In [28]:
all(arrs_eq_nan(np.array(raw1_all_pycbc[i]._data), raw1_all_pygwf[ch]) for i, ch in enumerate(raw1_chs_all))

True